In [2]:
import nest_asyncio
nest_asyncio.apply()

from dataclasses import dataclass
import asyncio
from agents.extensions.visualization import draw_graph
import os 
from openai import AsyncOpenAI 
from dotenv import load_dotenv  
load_dotenv() 
from agents import (
    Agent,
    Runner, 
    set_tracing_disabled,OpenAIChatCompletionsModel,enable_verbose_stdout_logging
)   
 

# enable_verbose_stdout_logging()
set_tracing_disabled(disabled=True)
API_KEY=os.getenv("GEMINI_API_KEY")

if  not API_KEY  :
    raise ValueError("Please set EXAMPLE_BASE_URL, EXAMPLE_API_KEY, EXAMPLE_MODEL_NAME via env var or code.")

 
client = AsyncOpenAI(base_url="https://generativelanguage.googleapis.com/v1beta/openai/",api_key=API_KEY,)

model = OpenAIChatCompletionsModel(model="gemini-2.0-flash",openai_client=client)

In [20]:
from agents import (
    Agent,InputGuardrailTripwireTriggered,
    Runner,
    set_tracing_disabled,
    ModelSettings,
    input_guardrail,
    RunContextWrapper,
    TResponseInputItem,
    GuardrailFunctionOutput,
)
from pydantic import BaseModel


class MathHomeworkOutput(BaseModel):
    is_math_homework: bool
    reasoning: str
enable_verbose_stdout_logging()

set_tracing_disabled(disabled=False)
guardrail_agent = Agent(
    name="Guardrail check",
    instructions="Check if the user is asking you to do their math homework.",
    output_type=MathHomeworkOutput,
    model=model,
)


@input_guardrail
def math_guardrail(
    ctx: RunContextWrapper[None], agent: Agent, input: str | list[TResponseInputItem]
) -> GuardrailFunctionOutput:
    result = Runner.run_sync(guardrail_agent, input, context=ctx.context)

    return GuardrailFunctionOutput(
        output_info=result.final_output,  # (3)!
        tripwire_triggered=result.final_output.is_math_homework,
    )


agent = Agent(  # (4)!
    name="Customer support agent",
    instructions="You are a customer support agent. You help customers with their questions.",
    input_guardrails=[math_guardrail],
    model=model,
)
try:
    resp = await Runner.run(agent, "what is ai",max_turns=1)
    
    print(resp.final_output)
    print("Guardrail didn't trip - this is unexpected")

except InputGuardrailTripwireTriggered:
    print("Math homework guardrail tripped")
# resp = await Runner.run(agent, "what is ai")
# print(resp.final_output)

Creating trace Agent workflow with id trace_4feba6089abe4e0f93fd1d1506e661b2
Creating trace Agent workflow with id trace_4feba6089abe4e0f93fd1d1506e661b2
Setting current trace: trace_4feba6089abe4e0f93fd1d1506e661b2
Setting current trace: trace_4feba6089abe4e0f93fd1d1506e661b2
Creating span <agents.tracing.span_data.AgentSpanData object at 0x0000019FA21D5630> with id None
Creating span <agents.tracing.span_data.AgentSpanData object at 0x0000019FA21D5630> with id None
Running agent Customer support agent (turn 1)
Running agent Customer support agent (turn 1)
Creating span <agents.tracing.span_data.GuardrailSpanData object at 0x0000019FA2235E10> with id None
Creating span <agents.tracing.span_data.GuardrailSpanData object at 0x0000019FA2235E10> with id None
Creating span <agents.tracing.span_data.AgentSpanData object at 0x0000019FA22225D0> with id None
Creating span <agents.tracing.span_data.AgentSpanData object at 0x0000019FA22225D0> with id None
Running agent Guardrail check (turn 1)
R

In [19]:
from agents import (
    Agent,InputGuardrailTripwireTriggered,
    Runner,
    set_tracing_disabled,
    ModelSettings,
    input_guardrail,
    RunContextWrapper,
    TResponseInputItem,
    GuardrailFunctionOutput,
)
from pydantic import BaseModel


class MathHomeworkOutput(BaseModel):
    is_math_homework: bool
    reasoning: str


set_tracing_disabled(disabled=False)
guardrail_agent = Agent(
    name="Guardrail check",
    instructions="Check if the user is asking you to do their math homework.",
    output_type=MathHomeworkOutput,
    model=model,
)


@input_guardrail
async def math_guardrail(
    ctx: RunContextWrapper[None], agent: Agent, input: str | list[TResponseInputItem]
) -> GuardrailFunctionOutput:
    print(f"input:{input}")
    print(f"agent name :{agent.name}")
    print(f"ctx :{ctx.context}")
    result = await Runner.run(guardrail_agent, input, context=ctx.context)

    return GuardrailFunctionOutput(
        output_info=result.final_output,  # (3)!
        tripwire_triggered=result.final_output.is_math_homework,
    )


agent = Agent(  # (4)!
    name="Customer support agent",
    instructions="You are a customer support agent. You help customers with their questions.",
    input_guardrails=[math_guardrail],
    model=model,
)
try:
    resp = await Runner.run(agent, "what is 3x+3x+4=4x",max_turns=1)
    print(resp.final_output)
    print("Guardrail didn't trip - this is unexpected")

except InputGuardrailTripwireTriggered:
    print("Math homework guardrail tripped")
# resp = await Runner.run(agent, "what is ai")
# print(resp.final_output)

Creating trace Agent workflow with id trace_56aa683971c44f9e9b88160ab1a6facf
Setting current trace: trace_56aa683971c44f9e9b88160ab1a6facf
Creating span <agents.tracing.span_data.AgentSpanData object at 0x0000019FA21A4F00> with id None
Running agent Customer support agent (turn 1)
Creating span <agents.tracing.span_data.GuardrailSpanData object at 0x0000019FA2227CD0> with id None
input:what is 3x+3x+4=4x
agent name :Customer support agent
ctx :None
Creating span <agents.tracing.span_data.AgentSpanData object at 0x0000019FA22039D0> with id None
Running agent Guardrail check (turn 1)
Creating span <agents.tracing.span_data.GenerationSpanData object at 0x0000019FA2209A90> with id None
[
  {
    "content": "You are a customer support agent. You help customers with their questions.",
    "role": "system"
  },
  {
    "role": "user",
    "content": "what is 3x+3x+4=4x"
  }
]
Tools:
[]
Stream: False
Tool choice: NOT_GIVEN
Response format: NOT_GIVEN

Creating span <agents.tracing.span_data.Gen

LLM resp:
{
  "content": "Okay, let's solve the equation step-by-step:\n\n1.  **Combine like terms:** On the left side of the equation, we have `3x + 3x`, which simplifies to `6x`. So the equation becomes:\n   `6x + 4 = 4x`\n\n2.  **Isolate the 'x' terms:** Subtract `4x` from both sides of the equation to get all the 'x' terms on one side:\n   `6x + 4 - 4x = 4x - 4x`\n   This simplifies to:\n   `2x + 4 = 0`\n\n3.  **Isolate the 'x' term:** Subtract 4 from both sides of the equation:\n   `2x + 4 - 4 = 0 - 4`\n   This simplifies to:\n   `2x = -4`\n\n4.  **Solve for 'x':** Divide both sides of the equation by 2:\n   `2x / 2 = -4 / 2`\n   This simplifies to:\n   `x = -2`\n\nSo, the solution to the equation `3x + 3x + 4 = 4x` is `x = -2`.",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}

